In [1]:
import numpy as np
import pandas as pd
import os
import re 

posts_full = pd.read_csv('../data/the-reddit-irl-dataset-posts.csv')
comments_full = pd.read_csv('../data/the-reddit-irl-dataset-comments.csv')

posts = posts_full.copy()
comments = comments_full.copy()

display(posts.head())
display(comments.head())

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ttdab1,2s5ti,meirl,False,1648771149,https://old.reddit.com/r/meirl/comments/ttdab1...,i.imgur.com,https://i.imgur.com/ucZiw34.jpg,NaN,meirl,93
1,post,ttd0r4,2s5ti,meirl,False,1648770296,https://old.reddit.com/r/meirl/comments/ttd0r4...,i.redd.it,https://i.redd.it/x8sehlq23tq81.jpg,NaN,Me_irl,1
2,post,ttd04u,2s5ti,meirl,False,1648770242,https://old.reddit.com/r/meirl/comments/ttd04u...,i.redd.it,https://i.redd.it/p86ehgxw2tq81.jpg,NaN,Meirl,605
3,post,ttc7og,2s5ti,meirl,False,1648767776,https://old.reddit.com/r/meirl/comments/ttc7og...,imgur.com,https://imgur.com/KIU84VZ.jpeg,NaN,meirl,67
4,post,ttc58g,2s5ti,meirl,False,1648767600,https://old.reddit.com/r/meirl/comments/ttc58g...,i.redd.it,https://i.redd.it/to2ot762vsq81.jpg,NaN,me irl,32


,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,comment,i2x2j0g,2s5ti,meirl,False,1648771186,https://old.reddit.com/r/meirl/comments/tswh3j...,Yes,NaN,1
1,comment,i2x2hqk,2s5ti,meirl,False,1648771171,https://old.reddit.com/r/meirl/comments/tt9v20...,It’s worse if you’re a man,-0.4767,5
2,comment,i2x2dv0,2s5ti,meirl,False,1648771121,https://old.reddit.com/r/meirl/comments/tt5aas...,They took our jerbs!!!!,0.0000,3
3,comment,i2x2d15,2s5ti,meirl,False,1648771110,https://old.reddit.com/r/meirl/comments/ttbnpo...,It certainly can help you from a mental standp...,0.7650,3
4,comment,i2x28i1,2s5ti,meirl,False,1648771053,https://old.reddit.com/r/meirl/comments/tswh3j...,Bottom picture,0.0000,1


In [2]:
display(posts.columns)
display(posts.shape)
display(comments.columns)
display(comments.shape)

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'domain', 'url', 'selftext', 'title',
       'score'],
      dtype='object')

(3362747, 12)

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'body', 'sentiment', 'score'],
      dtype='object')

(12053027, 10)

In [4]:
posts.loc[posts['id'] == 'tt1tbb']

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
135,post,tt1tbb,2s5ti,meirl,False,1648739180,https://old.reddit.com/r/meirl/comments/tt1tbb...,i.redd.it,NaN,[deleted],Meirl,13417


In [3]:
comments['id_post'] = comments.permalink.str.extract(r'https://old.reddit.com/r/meirl/comments/([^/]+)/meirl/.*', expand=False)
comments.rename(columns={"id": "id_comment"}, inplace=True)
comments.head()

,type,id_comment,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score,id_post
0,comment,i2x2j0g,2s5ti,meirl,False,1648771186,https://old.reddit.com/r/meirl/comments/tswh3j...,Yes,NaN,1,tswh3j
1,comment,i2x2hqk,2s5ti,meirl,False,1648771171,https://old.reddit.com/r/meirl/comments/tt9v20...,It’s worse if you’re a man,-0.4767,5,tt9v20
2,comment,i2x2dv0,2s5ti,meirl,False,1648771121,https://old.reddit.com/r/meirl/comments/tt5aas...,They took our jerbs!!!!,0.0000,3,tt5aas
3,comment,i2x2d15,2s5ti,meirl,False,1648771110,https://old.reddit.com/r/meirl/comments/ttbnpo...,It certainly can help you from a mental standp...,0.7650,3,ttbnpo
4,comment,i2x28i1,2s5ti,meirl,False,1648771053,https://old.reddit.com/r/meirl/comments/tswh3j...,Bottom picture,0.0000,1,tswh3j


In [4]:
posts_to_join = posts[['id', 'selftext', 'url', 'permalink', 'score']]
comments_to_join = comments[['id_post', 'body', 'sentiment', 'id_comment']]
comments_to_join = comments_to_join[(comments['body'].notnull()) & (comments['body'] != 'NaN')]

In [5]:
display(posts_to_join.columns)
display(posts_to_join.shape)
display(comments_to_join.columns)
display(comments_to_join.shape)

Index(['id', 'selftext', 'url', 'permalink', 'score'], dtype='object')

(3362747, 5)

Index(['id_post', 'body', 'sentiment', 'id_comment'], dtype='object')

(12052763, 4)

In [6]:
comments_with_posts = posts_to_join.set_index('id').join(comments_to_join.set_index('id_post'), how='inner')

In [7]:
comments_with_posts.head()

,selftext,url,permalink,score,body,sentiment,id_comment
2dlhff,NaN,http://imgur.com/0NHshtG,https://old.reddit.com/r/meirl/comments/2dlhff...,19,frig off,NaN,cjqz65u
2nfgu1,NaN,http://i.imgur.com/Ldg6D2y.jpg,https://old.reddit.com/r/meirl/comments/2nfgu1...,7,"I soar on an updraft, and am ever twirling, tw...",0.0000,cmdwd13
2nfgu1,NaN,http://i.imgur.com/Ldg6D2y.jpg,https://old.reddit.com/r/meirl/comments/2nfgu1...,7,this speaks to me like a plastic bag blowing i...,0.3612,cmdwat2
2rispp,NaN,http://i.imgur.com/qgdbTpB.gif,https://old.reddit.com/r/meirl/comments/2rispp...,31,Sauce?,NaN,cnjc52c
2rispp,NaN,http://i.imgur.com/qgdbTpB.gif,https://old.reddit.com/r/meirl/comments/2rispp...,31,Don't we all,0.0000,cngaib9


In [8]:
student_comments = comments_with_posts[(comments_with_posts['body'].str.contains('university', flags=re.IGNORECASE, regex=True)) |
                                     (comments_with_posts['body'].str.contains('college', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('student', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('teacher', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('professor', flags=re.IGNORECASE, regex=True))]
student_comments.index.name = 'id_post'
display(student_comments)

,selftext,url,permalink,score,body,sentiment,id_comment
id_post,,,,,,,
3zw04q,NaN,https://i.imgur.com/wgfQy9j.jpg,https://old.reddit.com/r/meirl/comments/3zw04q...,141,"Holy mother of buzzwords, this is worse than c...",-0.4767,cyq16uc
40m9es,[deleted],NaN,https://old.reddit.com/r/meirl/comments/40m9es...,4109,"Can confirm, born in 97 and am a freshman in c...",0.0000,cyvwmae
40m9es,[deleted],NaN,https://old.reddit.com/r/meirl/comments/40m9es...,4109,Someone born yesterday won't be entering colle...,0.0000,cyvv1p9
40m9es,[deleted],NaN,https://old.reddit.com/r/meirl/comments/40m9es...,4109,"Some kids born in '97 are in college now, so i...",0.0000,cyvuyg2
42fqkn,NaN,http://i.imgur.com/cx7Ak8a.jpg,https://old.reddit.com/r/meirl/comments/42fqkn...,801,Is that Brian Murphy from College humor?,0.2732,czajya7
...,...,...,...,...,...,...,...
tsln86,NaN,https://i.redd.it/edpns7wtplq81.jpg,https://old.reddit.com/r/meirl/comments/tsln86...,3371,"Yes because stay at home being a nurse, elderl...",-0.2163,i2tmp56
tsn173,[deleted],NaN,https://old.reddit.com/r/meirl/comments/tsn173...,129,So true teachers hate on us rapers i dont get ...,-0.5520,i2tk8is
tswh3j,NaN,https://i.redd.it/lxhymxn11pq81.jpg,https://old.reddit.com/r/meirl/comments/tswh3j...,143559,Once had the best train ride sitting next to a...,0.6124,i2vtmj5


In [9]:
student_comments['has_more_than_one_comment'] = (student_comments[student_comments['url'].notnull()].groupby(by='id_post')['id_comment'].count() > 1)


/var/folders/1f/_q0hr5gj30325cjg2f37b6q80000gq/T/ipykernel_10667/3961217209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_comments['has_more_than_one_comment'] = (student_comments[student_comments['url'].notnull()].groupby(by='id_post')['id_comment'].count() > 1)


In [10]:
comments_to_extract = student_comments[(student_comments['has_more_than_one_comment']) & (student_comments['url'].notnull())]
comments_to_extract.head()

,selftext,url,permalink,score,body,sentiment,id_comment,has_more_than_one_comment
id_post,,,,,,,,
44llao,NaN,http://i.imgur.com/YdvXrdD.jpg,https://old.reddit.com/r/meirl/comments/44llao...,7729,No that's pretty standard. Profanity (indirect...,-0.5897,d2br00g,True
44llao,NaN,http://i.imgur.com/YdvXrdD.jpg,https://old.reddit.com/r/meirl/comments/44llao...,7729,Oh definitely but that's English schooling for...,-0.4854,czs05du,True
44llao,NaN,http://i.imgur.com/YdvXrdD.jpg,https://old.reddit.com/r/meirl/comments/44llao...,7729,Around the time of South Park I got suspended ...,-0.5923,czrklfa,True
44qzad,NaN,https://i.imgur.com/t7FtM5o.gif,https://old.reddit.com/r/meirl/comments/44qzad...,3767,Student loans and the belief that nothing in l...,0.6028,czu1fjo,True
44qzad,NaN,https://i.imgur.com/t7FtM5o.gif,https://old.reddit.com/r/meirl/comments/44qzad...,3767,What do you mean? There's a university and a b...,0.0000,cztiyzi,True


In [11]:
from random import sample
post_with_student_comments_ids = sample(list(comments_to_extract.index.unique()), 15)


In [12]:
post_with_student_comments_ids

['mwqoll',
 'fd6ygc',
 'js6h2h',
 'p55wvp',
 'g1phm6',
 '66prls',
 'tj4i58',
 '6skobw',
 '7r3zu0',
 '44llao',
 '718rwe',
 'j8fxde',
 'g244o8',
 'l0klw0',
 'l02ti3']

In [13]:
student_comments_ids = sample(list(comments_to_extract[comments_to_extract.index.isin(post_with_student_comments_ids)].id_comment), 30)

In [14]:
student_comments_ids

['dley49l',
 'fnhtuzu',
 'dn9hb9j',
 'fnhug6l',
 'dglhwq2',
 'h97d9px',
 'dsvt2ns',
 'gvjym5s',
 'fjh1kph',
 'dgkrtsz',
 'dgkhukx',
 'fjgq6h9',
 'dgm76yw',
 'gjvjltm',
 'gvkl4mr',
 'g8d89yg',
 'dn9i9q0',
 'gjvddta',
 'h952zoj',
 'dn96cmc',
 'g8cn1nz',
 'dsuyqwm',
 'gk0mv8l',
 'i1j1fnk',
 'gvkejip',
 'h94ykdg',
 'h9456w4',
 'gvjy08t',
 'gvlq4zl',
 'g2dk7o2']

In [16]:
sample_posts = posts_full[posts_full.id.isin(post_with_student_comments_ids)]
sample_comments = comments_full[comments_full.id.isin(student_comments_ids)]

In [17]:
sample_posts

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
1846,post,tj4i58,2s5ti,meirl,False,1647841019,https://old.reddit.com/r/meirl/comments/tj4i58...,i.redd.it,https://i.redd.it/rguvxweuboo81.jpg,NaN,Meirl,75420
24756,post,p55wvp,2s5ti,meirl,False,1629076220,https://old.reddit.com/r/meirl/comments/p55wvp...,i.redd.it,https://i.redd.it/ngjhe18hemh71.jpg,NaN,meirl,5320
34378,post,mwqoll,2s5ti,meirl,False,1619166594,https://old.reddit.com/r/meirl/comments/mwqoll...,i.redd.it,https://i.redd.it/gxgoy3t1wvu61.jpg,NaN,meirl,30758
47998,post,l0klw0,2s5ti,meirl,False,1611065661,https://old.reddit.com/r/meirl/comments/l0klw0...,i.imgur.com,https://i.imgur.com/Fc4mQX4.jpg,NaN,meirl,47016
48116,post,l02ti3,2s5ti,meirl,False,1611001139,https://old.reddit.com/r/meirl/comments/l02ti3...,i.redd.it,https://i.redd.it/f1i01bhzf5c61.jpg,NaN,Meirl,5073
56295,post,js6h2h,2s5ti,meirl,False,1605093964,https://old.reddit.com/r/meirl/comments/js6h2h...,i.redd.it,https://i.redd.it/jf7zp65wily51.png,NaN,meirl,18547
60486,post,j8fxde,2s5ti,meirl,False,1602312786,https://old.reddit.com/r/meirl/comments/j8fxde...,i.imgur.com,https://i.imgur.com/CPYqr5c.jpg,NaN,Meirl,191
84475,post,g244o8,2s5ti,meirl,False,1586995854,https://old.reddit.com/r/meirl/comments/g244o8...,i.redd.it,https://i.redd.it/5j5n84twn2t41.jpg,NaN,meirl,227
84557,post,g1phm6,2s5ti,meirl,False,1586946865,https://old.reddit.com/r/meirl/comments/g1phm6...,i.redd.it,https://i.redd.it/8ot2cgg9mys41.jpg,NaN,meirl,26111
90629,post,fd6ygc,2s5ti,meirl,False,1583292870,https://old.reddit.com/r/meirl/comments/fd6ygc...,i.redd.it,https://i.redd.it/d7ixzl22tkk41.jpg,NaN,meirl,13027


In [18]:
sample_comments

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
34645,comment,i1j1fnk,2s5ti,meirl,False,1647869871,https://old.reddit.com/r/meirl/comments/tj4i58...,This reminds me when I was in first school and...,0.5106,1
445897,comment,h97d9px,2s5ti,meirl,False,1629146540,https://old.reddit.com/r/meirl/comments/p55wvp...,I had a dream about one of my favorite teacher...,0.3400,2
446241,comment,h952zoj,2s5ti,meirl,False,1629110177,https://old.reddit.com/r/meirl/comments/p55wvp...,I was a math kid so I got buddy buddy with all...,-0.4347,5
446261,comment,h94ykdg,2s5ti,meirl,False,1629106621,https://old.reddit.com/r/meirl/comments/p55wvp...,Thats the worst thing ever. one time I gotot y...,-0.5719,10
446414,comment,h9456w4,2s5ti,meirl,False,1629085190,https://old.reddit.com/r/meirl/comments/p55wvp...,Then you got the opposite when you get praised...,0.7351,89
520491,comment,gvlq4zl,2s5ti,meirl,False,1619207030,https://old.reddit.com/r/meirl/comments/mwqoll...,That comes across as a pretty leading question...,-0.9543,3
520804,comment,gvkl4mr,2s5ti,meirl,False,1619189112,https://old.reddit.com/r/meirl/comments/mwqoll...,&gt;Having children is the most destructive th...,-0.2716,1
520873,comment,gvkejip,2s5ti,meirl,False,1619186114,https://old.reddit.com/r/meirl/comments/mwqoll...,Agricultural scientist here. There have been s...,-0.8792,4
521128,comment,gvjym5s,2s5ti,meirl,False,1619176756,https://old.reddit.com/r/meirl/comments/mwqoll...,"They *donated* $70,200 to him right when he di...",-0.8827,10
521136,comment,gvjy08t,2s5ti,meirl,False,1619176290,https://old.reddit.com/r/meirl/comments/mwqoll...,What about when they funded the guy (Coronado?...,-0.7149,9


In [19]:
sample_posts.to_csv('../data/sample_posts.csv', index=False)
sample_comments.to_csv('../data/sample_comments.csv', index=False)